In [ ]:
#| default_exp testing

%pip install -q langchain_core
%pip install -q langchain_openai
%pip install -q psycopg2-binary
%pip install -q nbdev


In [ ]:

from langchain_core.prompts import PromptTemplate


In [ ]:
firstPrompt = PromptTemplate.from_template("""Return one ore more json objects together in an array to describe the jobs in a job posting. If there is only one job posting presented here, return a json object containing a summary of every separate piece of information in the job posting. If the job describes multiple roles, create separate json objects for each role. Here is the job posting:\n{job_posting}""")

subsequentPrompt = PromptTemplate.from_template("""You will be returning one ore more json objects together in an array to describe the jobs in a job posting. Here are examples of the format of the json object you should return:\n{json_examples}\nHere are the key values currently being used to categorize jobs:\n{key_values}.\n If there is only one job posting presented here, return a json object containing a summary of every separate piece of information in the job posting. If the job describes multiple roles, create a separate json object for each role. Use the existing key values where they are relevant. If there is no relevant existing key, add new, relevant key to your json object:\n{job_posting}""")



In [ ]:
prompt  = subsequentPrompt.format(job_posting="This is a job posting", json_examples="[{\"role\": \"Software Engineer\", \"location\": \"San Francisco\", \"salary\": \"$120,000\"}, {\"role\": \"Data Scientist\", \"location\": \"New York\", \"salary\": \"$150,000\"}]", key_values="role, location, salary")

print(prompt)


In [ ]:
import json

# Opening JSON file
f = open('../posts.json')

# returns JSON object as 
# a dictionary
posts = json.load(f)

In [ ]:
#| export
import psycopg2

def connect():
    conn = psycopg2.connect("dbname=Bumpant user=Bumpant password=ampegskb")
    cur = conn.cursor()
    return conn, cur

conn = psycopg2.connect("dbname=Bumpant user=Bumpant password=ampegskb")
cur = conn.cursor()

cur.execute(
    """CREATE TABLE IF NOT EXISTS parsedPosts (
        id SERIAL PRIMARY KEY, 
        hnuser TEXT, 
        date TIMESTAMP, 
        comment TEXT,
        embedding vector(1536)
        );"""
    )

conn.commit()    

In [ ]:
from langchain_openai import ChatOpenAI

post = posts[0]

msg = firstPrompt.format(job_posting = post['comment'])

model = ChatOpenAI()

response = model.invoke(msg)

In [ ]:
# Parse the JSON string to a Python dictionary
initial_parsed_datum = json.loads(response.content)
parsed_data = [ initial_parsed_datum ]

In [ ]:
import  json

result = {}
keys = []

examplesStr = ""
postItems = posts[:2]

for i in range(len(postItems)):
    if i != 0:
        msg = subsequentPrompt.format(job_posting = postItems[i]['comment'], json_examples = examplesStr, key_values = ", ".join(keys))
        
        print(msg)
        resp = model.invoke(msg)
        parsed_datum = json.loads(resp.content)
        
        for  job in  parsed_datum:
            parsed_data.append(job)

        print(parsed_data)


In [ ]:
parsed_data


In [ ]:
import  json

result = {}
keys = []

examplesStr = ""
postItems = posts[:2]

for i in range(len(postItems)):
    if i != 0:
        msg = subsequentPrompt.format(job_posting = postItems[i]['comment'], json_examples = examplesStr, key_values = ", ".join(keys))

        print(msg)
        resp = model.invoke(msg)
        parsed_datum = json.loads(resp.content)
        for  job in  parsed_datum:
            parsed_data.append(job)

        # We don't care much for the shape of the json object the machine generates, as long as it remains uniform. So, we use the first 5 outputs as examples for the rest.
        if i < 5:
            itemStr = map(lambda x: json.dumps(x), parsed_data)
            examplesStr += " ".join(itemStr)

        if i == 4 or i == 0:
            print('\n\n', msg)
            print ('\n\n', examplesStr)
        # We don't care much for the shape of the json object the machine generates, as long as it remains uniform. So, we use the first 5 outputs as examples for the rest.
        if i < 5:
            itemStr = map(lambda x: json.dumps(x), parsed_data)
            examplesStr += " ".join(itemStr)

        if i == 4 or i == 0:
            print('\n\n', msg)
            print ('\n\n', examplesStr)

        print(parsed_data)





We split the iteration over the request so we don't do it unnecessarily

In [ ]:

for i in range(len(parsed_data)):
        # keys needs to be a unique list of keys
        # so we can't just add the keys of the new datum
        # we need to add only the keys that are not already in keys

        jobdesc = parsed_data[i]
        for job in jobdesc:
            print(job)
            # parsed_data_keys = list(job.keys()) 
            # for key in parsed_data_keys:
            #     if key not in keys:
            #         keys.append(key)
            #         # curs.execute("ALTER TABLE parsedPosts ADD COLUMN IF NOT EXISTS %s TEXT", (key,))
            #         conn.commit()

            # keyStr = ", ".join(keys)

            # curs.execute("INSERT INTO posts (hnuser, date, comment, %s) VALUES (%s, %s, %s, %s)", (posts[i]['hnuser'], posts[i]['date'], posts[i]['comment'], parsed_datum))

In [ ]:
print(parsed_data)

In [ ]:

# let's see how frequently each key appears
tally = {}
keys
for key in keys:
#     result[key] = [datum[key] for datum in parsed_data]
    tally[key] = sum([1 for datum in parsed_data if key in datum])

# result, tally

In [ ]:
parsed_data

We add all the gathered keys to the table.

In [ ]:
for key in keys:
    cur.execute(f"ALTER TABLE parsedPosts ADD COLUMN IF NOT EXISTS  \"{key}\"  TEXT")
    conn.commit()

We create a SQL query that posts by our new keys

In [ ]:
sql_query = "INSERT INTO parsedPosts ("+ ", ".join(keys) +") VALUES (" + ", ".join(["%s" for key in keys]) + ")"

conn, cur = connect()

# for i in range(len(parsed_data)):
#     datum = parsed_data[i]
#     datum_keys = list(datum.keys())

#     for key in keys:
#         if key not in datum_keys:
#             datum[key] = None

#     values = [datum[key] for key in keys]
#     valuees = [str(value) for value in values]
#     print(values)
#     # cur.execute(sql_query, valuees)
#     # conn.commit()

cur.execute("SELECT * FROM parsedPosts")

cur.fetchall()

# cur.close()

In [1]:
#| hide
from nbdev import nbdev_export
nbdev_export()

<unknown>:3: SyntaxWarning: invalid escape sequence '\ '
